In [1]:
import pandas as pd
import numpy as np

In [2]:
####

workouts = pd.read_csv('workouts.csv')
payments = pd.read_excel('payments.xlsx')
users = pd.read_excel('users.xlsx')

workouts['start_at'] = pd.to_datetime(workouts['start_at'])
users['first_contact_datetime'] = pd.to_datetime(users['first_contact_datetime'])

In [3]:
workouts.head()

,workout_id,cost,start_at,status,workout_schedule_type,client_id,client_status,workout_type,trainer_department,trainer_id
0,30793909,NaN,2020-11-13 12:45:00,success,trial,22034,NaN,general,Sales,10722051
1,31123309,750.0,2020-11-21 13:00:00,success,regular,22034,new,general,Spartacus,940642
2,31412167,750.0,2020-11-28 13:00:00,success,regular,22034,new,general,Spartacus,940642
3,31703605,750.0,2020-12-05 13:00:00,success,regular,22034,active,general,Spartacus,940642
4,26904500,NaN,2020-08-06 18:20:00,success,trial,88101,NaN,general,Sales,3826530


In [4]:
payments.head()

,user_id,payment_id,workout_type,amount,payment_date
0,132815,1535249,general,10337.768848,2020-01-30 15:03:59
1,165987,2395447,general,9711.925350,2020-08-18 21:59:09
2,17364,2641443,general,8951.474487,2020-10-12 11:55:00
3,148617,2402771,general,9981.884731,2020-08-20 21:30:27
4,175413,2035719,general,8848.691858,2020-05-22 20:10:55


In [5]:
users.head()

,first_contact_datetime,age,workouts_successful,workouts_total,months_active,user_id,region,geo_group
0,2020-08-08 13:57:25,NaN,10,13,2,2790000,NaN,СНГ
1,2020-05-02 00:30:02,NaN,26,38,7,780106,NaN,СНГ
2,2019-06-27 13:10:33,30.0,27,33,6,1133376,NaN,СНГ
3,2020-04-22 15:37:58,22.0,59,59,8,1996499,NaN,СНГ
4,2016-06-23 16:21:40,24.0,8,12,2,57899,Москва и Московская область,Москва


### Запрос python соответствующий SQL

~~~ sql
select    workout_type
        , count(workout_id) as cnt
from workouts
group by workout_type
order by cnt desc
~~~

In [6]:
####

workouts.groupby('workout_type').agg(cnt = ('workout_id', 'count')).reset_index().sort_values('cnt', ascending = False)

,workout_type,cnt
2,general,103492
0,cycling,1175
1,functional test,26


### Запрос python соответствующий SQL

~~~ sql
select    date_part('month', start_at) as dm
        , trainer_department
        , count(workout_id) as cnt
from workouts
group by   dm
         , trainer_department
order by dm
~~~

In [7]:
####

workouts['dm'] = workouts['start_at'].dt.month
workouts.groupby(['dm', 'trainer_department']).agg(cnt = ('workout_id', 'count')).reset_index().sort_values('dm')

,dm,trainer_department,cnt
0,1,Athletic,11
1,1,Consultant,164
2,1,Cycling,1
3,1,Dinamo,36
4,1,Sales,457
...,...,...,...
78,12,Consultant,4
79,12,Cycling,77
80,12,Dinamo,254
81,12,Sales,22


### Запрос python соответствующий SQL

~~~ sql
select    date_trunc('month', payment_date) as p_tm
        , sum(amount)/1000 as sum_amt
from payments
where workout_type = 'general'
group by p_tm
order by p_tm
~~~

In [8]:
####

payments['p_tm'] = payments['payment_date'].dt.to_period('M').dt.to_timestamp()

result = payments[payments['workout_type'] == 'general'].groupby('p_tm').agg(sum_amt = ('amount', 'sum')).reset_index().sort_values('p_tm')
result['sum_amt'] = result['sum_amt'] /1000
round(result, 2)

,p_tm,sum_amt
0,2020-01-01,4370.77
1,2020-02-01,6114.60
2,2020-03-01,7949.12
3,2020-04-01,11908.99
4,2020-05-01,11125.84
5,2020-06-01,9682.46
6,2020-07-01,9553.44
7,2020-08-01,8345.36
8,2020-09-01,9876.67
9,2020-10-01,11044.86


### Запрос python соответствующий SQL

~~~ sql
select    user_id
        , min(date_trunc('day', payment_date)) as first_date
from payments p
    join users u
        on p.user_id = u.user_id
where region is not null
group by user_id
order by user_id
limit 20
~~~

In [9]:
####

payments['first_date'] = payments['payment_date'].dt.to_period('D').dt.to_timestamp()
new_df = payments.merge(users, on = 'user_id', how = 'inner')
new_df[~new_df['region'].isnull()].groupby('user_id').agg({'first_date':'min'}).reset_index().sort_values('user_id').head(20)

,user_id,first_date
0,185,2020-04-02
1,1455,2020-02-03
2,1558,2020-05-12
3,5312,2020-02-21
4,5804,2020-01-15
5,6877,2020-11-03
6,7226,2020-03-29
7,7824,2020-02-22
8,8881,2020-04-05
9,8913,2020-08-01


### Запрос python соответствующий SQL

~~~ sql
select    avg(age) as avg_age
from users p
    left join payments u
        on p.user_id = u.user_id
where payment_date is null
~~~

In [10]:
####

new_df1 = users.merge(payments, on = 'user_id', how = 'left')
new_df1[new_df1['payment_date'].isnull()].agg(avg_years_old = ('age', 'mean')).reset_index().rename(columns = {'index':'agregation_by'})

,agregation_by,age
0,avg_years_old,29.5


### Запрос python соответствующий SQL

~~~ sql
select   p_tm
       , sum(case when workout_type<>'general' then 1.0 else 0.0 end) / count(payment_id) as share_not_general
from payments
group by p_tm
~~~

In [11]:
####

payments['p_tm'] = payments['payment_date'].dt.month
payments['nogeneral_flag'] = np.where(payments['workout_type'] != 'general', 1, 0)
df2 = payments.groupby('p_tm').agg({'nogeneral_flag':'sum', 'payment_id':'count'}).reset_index().rename(columns = {'payment_id':'cnt_pay_id'})
df2['share_not_general'] = df2['nogeneral_flag'] / df2['cnt_pay_id']
df2

,p_tm,nogeneral_flag,cnt_pay_id,share_not_general
0,1,2,470,0.004255
1,2,8,662,0.012085
2,3,6,861,0.006969
3,4,9,1284,0.007009
4,5,4,1206,0.003317
5,6,11,1054,0.010436
6,7,8,1045,0.007656
7,8,7,916,0.007642
8,9,19,1087,0.017479
9,10,22,1233,0.017843


### Запрос python соответствующий SQL

~~~ sql
select    w.clientId
        , cnt_workout
        , amt
from (select    client_id
              , count(workout_id) as cnt_workout
     from workouts
     group by client_id
     ) w
    join (select      user_id
                    , sum(amount) as amt
          from payments
          group by user_id
          ) p
        on p.user_id = w.client_id
limit 10
~~~

In [12]:
####

selected_wkts = workouts.groupby('client_id').agg(cnt_workout = ('workout_id','count')).reset_index()
selected_paym = payments.groupby('user_id').agg(amt = ('amount','sum')).reset_index()
new_df3 = selected_wkts.merge(selected_paym, how = 'inner', left_on=['client_id'], right_on=
['user_id'])
new_df3 = new_df3.drop(columns = ['user_id'])
new_df3.head(10)

,client_id,cnt_workout,amt
0,185,50,29033.865003
1,1455,64,149649.891518
2,1558,13,18259.280262
3,5312,16,28416.673028
4,5804,46,38121.522478
5,6877,10,18460.888941
6,7226,18,27234.575055
7,7824,9,9177.146485
8,8881,52,28763.182307
9,8913,38,47770.522298


In [13]:
################################################################################################################################
################################################################################################################################
################################################################################################################################